# Construindo a Arquitetura da CNN 

## 1. Importando as bibliotecas

In [1]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Convolution1D, MaxPooling1D, Flatten, Dense, Dropout
from keras.preprocessing.image import load_img, img_to_array
from keras import utils
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import LabelEncoder
import numpy as np
import glob
from tqdm import tqdm
import re

## 2. Aquisição dos dados

In [2]:
dataset_path = 'C://Users//alexw//Documents//UFPI//Sistemas_Inteligentes//21-06-10_CNN//animals//'

In [3]:
def get_images_name_class(image_path):
    image = image_path.split('\\')[-1]
    image_name = re.sub(r'\.jpg', "", image)
    
    class_name = image_path.split('\\')[-2]
    
    return image_name, class_name

In [4]:
def run(dataset_path):
    #Captura os nomes das pastas
    name_folders = glob.glob(dataset_path + '*')
    cats_labels, cats_images = [],[] # armazena as imagens e os labels da classe gatos
    dogs_labels, dogs_images = [],[] # armazena as imagens e os labels da classe dogs
    panda_labels, panda_images = [],[] # armazena as imagens e os labels da classe panda
    
    for names_ in name_folders:
        #Captura o caminho completo da imagem
        path_images = glob.glob(names_ + '\\*.jpg')
        class_ = names_.split('\\')[-1]
        
        for i,path_ in enumerate(tqdm(path_images, desc=class_)):
            image_name, label = get_images_name_class(path_)
            image = load_img(path_, 
                             color_mode='rgb', 
                             target_size=(64,64), 
                             interpolation='nearest') # Carerga a imagem em formato RGB, com resolução 64x64
            
            if label == "cats":
                cats_labels.append(label)
                img_arr = img_to_array(image)
                cats_images.append(img_arr)
                
            elif label == "dogs":
                dogs_labels.append(label)
                img_arr = img_to_array(image)
                dogs_images.append(img_arr)
                
            elif label == "panda":
                panda_labels.append(label)
                img_arr = img_to_array(image)
                panda_images.append(img_arr) 
  
    # divide classe cats em 80% treino e 20% teste com embaralhamaento ativado
    cats_xtrain, cats_xtest, cats_ytrain, cats_ytest = train_test_split(cats_images, 
                                                                          cats_labels, 
                                                                          train_size=0.8, 
                                                                          shuffle=True) 
    # divide classe dogs em 80% treino e 20% teste com embaralhamaento ativado
    dogs_xtrain, dogs_xtest, dogs_ytrain, dogs_ytest = train_test_split(dogs_images, 
                                                                          dogs_labels, 
                                                                          train_size=0.8, 
                                                                          shuffle=True) 
    # divide classe panda em 80% treino e 20% teste com embaralhamaento ativado
    panda_xtrain, panda_xtest, panda_ytrain, panda_ytest = train_test_split(panda_images, 
                                                                               panda_labels, 
                                                                               train_size=0.8, 
                                                                               shuffle=True) 
    
    # unifica e converte para numpy array os x_train, x_test, y_train e y_test que foram divididos por classe
    x_tr = np.array(cats_xtrain + dogs_xtrain + panda_xtrain)
    x_te = np.array(cats_xtest + dogs_xtest + panda_xtest)
    y_tr = np.array(cats_ytrain + dogs_ytrain + panda_ytrain)
    y_te = np.array(cats_ytest + dogs_ytest + panda_ytest)
    
    return x_tr, x_te, y_tr, y_te
        

In [5]:
x_train, x_test, y_train, y_test = run(dataset_path)

panda: 100%|█████████████████████████████████████████████████████████████| 1000/1000 [00:11<00:00, 90.08it/s]


In [6]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(2400, 64, 64, 3)
(2400,)
(600, 64, 64, 3)
(600,)


## 3. Pré-processamento

In [7]:
x_train = x_train / 255.
x_test = x_test / 255.
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 3)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 3)

y_train = LabelEncoder().fit_transform(y_train)
y_train = utils.to_categorical(y_train)
y_test = LabelEncoder().fit_transform(y_test)
y_test = utils.to_categorical(y_test)


In [8]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(2400, 64, 64, 3)
(2400, 3)
(600, 64, 64, 3)
(600, 3)


## 4. Arquitetura da CNN

In [9]:
#Inicializando a CNN
classifier = Sequential()

#Camada de convolução
classifier.add(Convolution2D(32, kernel_size=(5,5), input_shape = (64, 64,3), activation = 'relu', padding='same', name = 'conv_1'))

#Camada de pooling
classifier.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2), padding='same', name = 'pool_1'))

#Segunda camada convolucional
classifier.add(Convolution2D(32, kernel_size=(5,5), activation = 'relu', padding='same', name = 'conv_2'))

#Camada de pooling
classifier.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2), padding='same', name = 'pool_2'))

#Segunda camada convolucional
classifier.add(Convolution2D(64, kernel_size=(5,5), activation = 'relu', padding='same', name = 'conv_3'))

#Dropout
classifier.add(Dropout(0.25))

#Segunda camada de pooling
classifier.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same', name = 'pool_3'))

#Vetorizando os mapas de características do último pooling (camada de entrada)
classifier.add(Flatten())

#Dropout
classifier.add(Dropout(0.5))

#Camada totalmente conectada ou oculta
classifier.add(Dense(activation='relu', units=128, name = 'dense_1'))


#Camada de saída
classifier.add(Dense(activation='softmax', units=3,  name = 'classification'))

In [10]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_1 (Conv2D)              (None, 64, 64, 32)        2432      
_________________________________________________________________
pool_1 (MaxPooling2D)        (None, 32, 32, 32)        0         
_________________________________________________________________
conv_2 (Conv2D)              (None, 32, 32, 32)        25632     
_________________________________________________________________
pool_2 (MaxPooling2D)        (None, 16, 16, 32)        0         
_________________________________________________________________
conv_3 (Conv2D)              (None, 16, 16, 64)        51264     
_________________________________________________________________
dropout (Dropout)            (None, 16, 16, 64)        0         
_________________________________________________________________
pool_3 (MaxPooling2D)        (None, 8, 8, 64)          0

## 5. Treinando o modelo

In [143]:
#Parâmetros de treinamento
epochs = 40
batch_size = 40
validation_split=0.1

In [144]:
classifier.compile(optimizer = 'adamax', loss= 'categorical_crossentropy', metrics=['accuracy'])

checkpoint = keras.callbacks.ModelCheckpoint('best_model.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='auto', save_freq='epoch') 
earlystop = keras.callbacks.EarlyStopping(patience=15)

In [145]:
classifier.fit(x_train, y_train, validation_split=validation_split, batch_size=batch_size, epochs=epochs, callbacks=[checkpoint,earlystop], verbose=1)

Epoch 1/40
54/54 [==============================] - 18s 319ms/step - loss: 1.0706 - accuracy: 0.4134 - val_loss: 0.9831 - val_accuracy: 0.6458

Epoch 00001: val_loss improved from inf to 0.98308, saving model to best_model.h5
Epoch 2/40
54/54 [==============================] - 18s 333ms/step - loss: 0.8305 - accuracy: 0.5653 - val_loss: 0.4109 - val_accuracy: 0.9292

Epoch 00002: val_loss improved from 0.98308 to 0.41093, saving model to best_model.h5
Epoch 3/40
54/54 [==============================] - 20s 370ms/step - loss: 0.7558 - accuracy: 0.6027 - val_loss: 0.5451 - val_accuracy: 0.8292

Epoch 00003: val_loss did not improve from 0.41093
Epoch 4/40
54/54 [==============================] - 19s 357ms/step - loss: 0.7196 - accuracy: 0.6174 - val_loss: 1.0333 - val_accuracy: 0.6167

Epoch 00004: val_loss did not improve from 0.41093
Epoch 5/40
54/54 [==============================] - 21s 381ms/step - loss: 0.7115 - accuracy: 0.6371 - val_loss: 0.4795 - val_accuracy: 0.8458

Epoch 0000

## 6. Avaliando o modelo

### Melhor wesultado item 1

In [15]:
best_model_1 = keras.models.load_model("75_best_model.h5")

In [16]:
score1 = best_model_1.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score1[0])
print("Test accuracy:", score1[1])

Test loss: 0.5844899415969849
Test accuracy: 0.9283333420753479


### Melhor resultado item 2

In [17]:
best_model2 = keras.models.load_model("77_2_best_model.h5") # ITEM 2

In [18]:
score2 = best_model2.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score2[0])
print("Test accuracy:", score2[1])

Test loss: 0.3047352731227875
Test accuracy: 0.9383333325386047
